<a href="https://colab.research.google.com/github/COFAlumni-USB/Physics-REBoot-Applied-Science-Bootcamp/blob/main/Presentaciones_Ponentes/Dia%204/JoseMedina_Regresion_Lineal_Multiple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresión Lineal Múltiple

Este es el notebook de la MLR que realizamos durante la sesión. Aquí la idea es conocer como es el proceso de crear un algoritmo, reconocer funciones importantes de sklearn. Entre los conceptos tenemos:

- Regresión Lineal: método estadístico en el que buscamos la recta y_pred = b0 + b1*x1 que mejor se ajuste a nuestro data set. SciKit Learn realiza este método utilizando mínimos cuadrados para reducir el error entre la recta y las observaciones (records.
- Regresión lineal múltiple: lo mismo pero en N dimensiones. Se busca el hiperplano b0 + b1x1 + ... = bnxn que menjor se ajusta a nuestro set de datos
- Dummy variable: Es una variable númerica que representa a una variable categórica para poder correr un método numérico que considere estas variables
- One Hot Encoder: un método para convertir cada variable categórica en un set de variables binarias que representen a las N categorías.
- Dummy Variable Trap: es el error de utilizar las N variables binarias de una categoría que esté one-hot-encoded. Utilizarlas todas introduce multicolinealidad al modelo y produce ruido en el resultado. Para evitarlo solo hay que usar N-1 dummies en el modelo
- Data preparation, data preparation, data preparation: la data debe estar lo más bella y procesada posible. Recuerden: garbage in, garbage out

In [ ]:
# Importamos las librerias necesarias

import numpy as np
import pandas as pd

In [ ]:
# Traer nuestro dataset
dataset = pd.read_csv("50_Startups.csv")

## Exploremos en Data set un poco. 

Es importante conocer la data con la que estamos trabajando. Hacer AI es mucho más que lanzar algorítmos, entender el problema, conocer la data que tengo a la mano y más importante.

In [ ]:
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [ ]:
dataset.describe()

,R&D Spend,Administration,Marketing Spend,Profit
count,50.000000,50.000000,50.000000,50.000000
mean,73721.615600,121344.639600,211025.097800,112012.639200
std,45902.256482,28017.802755,122290.310726,40306.180338
min,0.000000,51283.140000,0.000000,14681.400000
25%,39936.370000,103730.875000,129300.132500,90138.902500
50%,73051.080000,122699.795000,212716.240000,107978.190000
75%,101602.800000,144842.180000,299469.085000,139765.977500
max,165349.200000,182645.560000,471784.100000,192261.830000


## Preparemos la data

Aquí primero separamos el Data Frame original, que contiene todos los datos, en nuestra matriz de parámetros y nuestro vector de etiquetas.

In [ ]:
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

In [ ]:
X

array([[165349.2, 136897.8, 471784.1, 'New York'],
       [162597.7, 151377.59, 443898.53, 'California'],
       [153441.51, 101145.55, 407934.54, 'Florida'],
       [144372.41, 118671.85, 383199.62, 'New York'],
       [142107.34, 91391.77, 366168.42, 'Florida'],
       [131876.9, 99814.71, 362861.36, 'New York'],
       [134615.46, 147198.87, 127716.82, 'California'],
       [130298.13, 145530.06, 323876.68, 'Florida'],
       [120542.52, 148718.95, 311613.29, 'New York'],
       [123334.88, 108679.17, 304981.62, 'California'],
       [101913.08, 110594.11, 229160.95, 'Florida'],
       [100671.96, 91790.61, 249744.55, 'California'],
       [93863.75, 127320.38, 249839.44, 'Florida'],
       [91992.39, 135495.07, 252664.93, 'California'],
       [119943.24, 156547.42, 256512.92, 'Florida'],
       [114523.61, 122616.84, 261776.23, 'New York'],
       [78013.11, 121597.55, 264346.06, 'California'],
       [94657.16, 145077.58, 282574.31, 'New York'],
       [91749.16, 114175.79, 29491

In [ ]:
y

array([192261.83, 191792.06, 191050.39, 182901.99, 166187.94, 156991.12,
       156122.51, 155752.6 , 152211.77, 149759.96, 146121.95, 144259.4 ,
       141585.52, 134307.35, 132602.65, 129917.04, 126992.93, 125370.37,
       124266.9 , 122776.86, 118474.03, 111313.02, 110352.25, 108733.99,
       108552.04, 107404.34, 105733.54, 105008.31, 103282.38, 101004.64,
        99937.59,  97483.56,  97427.84,  96778.92,  96712.8 ,  96479.51,
        90708.19,  89949.14,  81229.06,  81005.76,  78239.91,  77798.83,
        71498.49,  69758.98,  65200.33,  64926.08,  49490.75,  42559.73,
        35673.41,  14681.4 ])

## Preparamos las variable categoricas con One Hot Encoding

La columna de estados pasa a ser tres columnas con valores 0 o 1 que representen a un estado o el otro.

New York = [0, 0 , 1]
Florida = [0, 1, 0]
California = [1, 0, 0]

Beware the dummy variable trap

In [ ]:
# One Hot Encoding

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3])], remainder='passthrough')

X = np.array(ct.fit_transform(X))

## Entrenamos nuestro modelo utilizando SciKit Learn

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## Evaluamos el performance de nuestro modelo

In [ ]:
y_test

array([103282.38, 144259.4 , 146121.95,  77798.83, 191050.39, 105008.31,
        81229.06,  97483.56, 110352.25, 166187.94])

In [ ]:
y_pred = regressor.predict(X_test)
print(y_pred)

[103015.20159795 132582.27760816 132447.73845175  71976.09851258
 178537.48221057 116161.24230167  67851.69209676  98791.73374687
 113969.43533014 167921.06569552]


## Con nuestro modelo listo podemos definir los coeficientes del hiperplano y evaluar cuan importante es un predictor respecto al otro.

En regresiones lineales, se puede interpretar la influencia de los predictores al leer el signo y el modulo de los coeficientes

In [ ]:
# Calcular los coeficientes

print(regressor.coef_)
print(regressor.intercept_)

[ 8.66383692e+01 -8.72645791e+02  7.86007422e+02  7.73467193e-01
  3.28845975e-02  3.66100259e-02]
42467.52924853204


## Predecimos cuan profitable será una empresa nueva

Ubicada en NY

Inversión en R&D $120000

Inversión en administración $78000

Inversión en Marketing $310000

In [ ]:
print(regressor.predict([[0.0, 0.0, 1.0, 120000, 78000, 310000]]))

[149983.70642383]
